# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from configparser import ConfigParser
config = ConfigParser()
config.read('../api_keys.cfg')

api_key = config.get('google', 'g_key')

### Store Part I results into DataFrame
* Load the csv exported in Part I (WeatherPy) to a DataFrame

In [30]:
weather_df = pd.read_csv('../output_data/weatherpy_cities.csv', encoding = 'utf-8')

#Visualise the dataframe
weather_df.head()

,City ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Chui,-33.6971,-53.4616,15.89,89,100,6.81,UY,1647652775
1,1,Hithadhoo,-0.6000,73.0833,28.40,74,100,7.42,MV,1647652579
2,2,Richards Bay,-28.7830,32.0377,18.26,85,9,1.92,ZA,1647652776
3,3,Ahipara,-35.1667,173.1667,20.66,77,27,2.72,NZ,1647652777
4,4,Punta Arenas,-53.1500,-70.9167,5.62,65,0,2.06,CL,1647652551


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = api_key)

In [15]:
#Store Lat and Lon into DF 
locations = weather_df[['Latitude', 'Longitude']]
# locations.head()

#Convert Humidity to float
humidity = weather_df["Humidity"].astype(float)

#Plot heatmap
fig1 = gmaps.figure(center = (21, 20), zoom_level =1.8)

#Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                   dissipating = False, max_intensity = 100, 
                                    point_radius = 3)

fig1.add_layer(heatmap_layer)
fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

My ideal weather conditions are:
* Temp between 16C - 23C
* Humidity between 30% and 50%
* Cloudiness percentage between 0-40%

In [22]:
ideal_weather_df = weather_df.loc[(weather_df['Max Temp'] >=16) & (weather_df["Max Temp"]<=23) 
                                 & (weather_df["Humidity"]>=30) & (weather_df["Humidity"]<=50) 
                                 & (weather_df["Cloudiness"]>=0) & (weather_df["Cloudiness"]<=40)]

ideal_weather_df.reset_index(drop = True)

,City ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,12,New Norfolk,-42.7826,147.0587,18.73,36,6,1.07,AU,1647652553
1,256,Laguna,38.4210,-121.4238,22.81,38,0,3.09,US,1647652797
2,365,Bandar-e Lengeh,26.5579,54.8807,20.12,47,0,6.34,IR,1647653113
3,390,Bārmer,25.7500,71.3833,21.66,45,0,3.26,IN,1647653135
4,409,Ukiah,39.1502,-123.2078,20.24,46,0,3.60,US,1647653156
5,417,San Patricio,28.0170,-97.5169,21.94,40,14,4.32,US,1647653162
6,418,Valparaíso,-33.0393,-71.6273,19.15,49,0,3.60,CL,1647653163


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = ideal_weather_df.loc[:,['City','Latitude','Longitude','Country']]

#Add Hotel Name column
hotel_df['Hotel Name'] = ""

hotel_df

,City,Latitude,Longitude,Country,Hotel Name
12,New Norfolk,-42.7826,147.0587,AU,
256,Laguna,38.4210,-121.4238,US,
365,Bandar-e Lengeh,26.5579,54.8807,IR,
390,Bārmer,25.7500,71.3833,IN,
409,Ukiah,39.1502,-123.2078,US,
417,San Patricio,28.0170,-97.5169,US,
418,Valparaíso,-33.0393,-71.6273,CL,


In [25]:
#Define base URL
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#Create params dictionary to update with each iteration
params = {"radius": 5000, 
         "types":"hotel", 
         "keyword":"hotel", 
         "key":api_key
         }

#Search for hotels within 5000 metres - set params using loop
for index, row in hotel_df.iterrows():
    
    #Get latitude and longitude as target coordinates
    lat = row["Latitude"]
    lon = row["Longitude"]
    
    #Add location to params using lat and lon
    params["location"] = f'{lat},{lon}'
    
    #make request to API
    hotels = requests.get(base_url, params = params).json()
    
    #Create try/except in case there's missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field. Skipping...")
        
hotel_df

,City,Latitude,Longitude,Country,Hotel Name
12,New Norfolk,-42.7826,147.0587,AU,Junction Motel
256,Laguna,38.4210,-121.4238,US,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
365,Bandar-e Lengeh,26.5579,54.8807,IR,Diplomat Hotel
390,Bārmer,25.7500,71.3833,IN,Hotel Kailash International
409,Ukiah,39.1502,-123.2078,US,Hampton Inn Ukiah
417,San Patricio,28.0170,-97.5169,US,"Motel 6 Sinton, TX"
418,Valparaíso,-33.0393,-71.6273,CL,Bo Hotel & Terraza


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [29]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig1.add_layer(hotel_markers)

# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))